# Pretraining GPT

## Mô tả bài toán:
Xây dựng bộ tách từ và mô hình cho bước tiền huấn luyện dựa trên kiến trúc của mô hình GPT2
* Thay thế code vào các đoạn `None` tương tự mẫu bên dưới để trả lời các câu hỏi.
* Không được thay đổi code ở các đoạn ngoài vùng yêu cầu.

```python
# START YOUR CODE
a = None
b = None
# END YOUR CODE
```

In [1]:
# Install libs
! pip install transformers==4.51.3 datasets==3.6.0 peft==0.15.2 huggingface_hub==0.31.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.2/484.2 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

ds = load_dataset("thainq107/c4-small")
ds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/388 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/118M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/13.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 10000
    })
})

### **C1 - Xây dựng bộ từ điển**

**Xây dựng bộ từ điển**
* Kích thước bộ từ điển tối đa: 50257
* Tần suất xuất hiện tối thiểu để token được đưa vào từ vựng: 50
* Danh sách các token đặc biệt: ["pad", "unk", "\<s\>"]

In [3]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.normalizers import NFKC
from tokenizers.decoders import ByteLevel as ByteLevelDecoder

# Initialize BPE tokenizer
tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = ByteLevel()
tokenizer.normalizer = NFKC()
tokenizer.decoder = ByteLevelDecoder()

################### START YOUR CODE ###################
trainer = BpeTrainer(
    vocab_size=50257,
    special_tokens=["<s>", "<pad>", "<unk>"]
)
################### END YOUR CODE ###################

tokenizer.train_from_iterator(ds["train"]["text"], trainer)
tokenizer.save("gpt_tokenizer.json")

In [4]:
# load tokenizer
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(tokenizer_file="gpt_tokenizer.json")
tokenizer.add_special_tokens({
    "bos_token": "<s>",
    "unk_token": "<unk>",
    "pad_token": "<pad>",
})
tokenizer.save_pretrained("gpt-tokenizer")

('gpt-tokenizer/tokenizer_config.json',
 'gpt-tokenizer/special_tokens_map.json',
 'gpt-tokenizer/tokenizer.json')

In [5]:
len(tokenizer)

50257

### **C2 - Xây dựng mô hình**

**Trong phần này, bạn sẽ định nghĩa mô hình GPT-2 dựa vào thư viện huggingface.**

* Bước 1: Tạo đối tượng GPT2Config:

Khi khởi tạo đối tượng GPT2Config với các tham số như sau:
- vocab_size: Lấy giá trị này từ thuộc tính vocab_size của đối tượng tokenizer bạn đã tạo ở phần trước.
- n_positions: Đặt giá trị là 512.
- n_ctx: Đặt giá trị là 512.
- n_embd: Đặt giá trị là 512.
- n_layer: Đặt giá trị là 24.
- n_head: Đặt giá trị là 16.
- bos_token_id: Lấy giá trị này từ thuộc tính

* Bước 2: Khởi tạo mô hình GPT2LMHeadModel:

Sử dụng đối tượng vừa tạo ở trên để khởi tạo một mô hình GPT2LMHeadModel.

Lưu ý: Mô hình được tạo theo cách này sẽ có các trọng số của nó sẽ được khởi tạo ngẫu nhiên (chưa được huấn luyện).

In [6]:
from transformers import GPT2Config, GPT2LMHeadModel

################### START YOUR CODE ###################
config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_positions=1024,
    n_ctx=1024,
    n_embd=1024,
    n_layer=24,
    n_head=16,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
)
################### END YOUR CODE ###################

model = GPT2LMHeadModel(config)

In [7]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Tổng số tham số: {total_params:,}")
print(f"Tham số huấn luyện được: {trainable_params:,}")

Tổng số tham số: 354,823,168
Tham số huấn luyện được: 354,823,168


### **C3 - Sinh văn bản**

In [8]:
import torch

def inference(sentence):
    inputs = tokenizer(sentence, return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        ################### START YOUR CODE ###################
        outputs = model(**inputs)

        ################### END YOUR CODE ###################
        logits = outputs.logits
    return logits

sentence = "I am delving into ancient history to write my thesis"
logits = inference(sentence)
print(logits.shape[1])

11


###**Lưu model lên Huggingface ở cài đặt chế độ private**
Model này sẽ được sử dụng để chạy code cho phần bài tập về PEFT

In [9]:
# login huggingface (use private token)
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `push_to_hub` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [10]:
model_name = "TSunm/gpt2-exam"
model.push_to_hub(model_name, private=True)
tokenizer.push_to_hub(model_name, private=True)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/TSunm/gpt2-exam/commit/076987c220100037d14e27256169534b420c0397', commit_message='Upload tokenizer', commit_description='', oid='076987c220100037d14e27256169534b420c0397', pr_url=None, repo_url=RepoUrl('https://huggingface.co/TSunm/gpt2-exam', endpoint='https://huggingface.co', repo_type='model', repo_id='TSunm/gpt2-exam'), pr_revision=None, pr_num=None)